In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import ttk  
import numpy as np
import librosa
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from scipy.io import wavfile
from keras.models import load_model
import sounddevice as sd
import soundfile as sf 
import time
# Load the model
model = load_model('model/SER_model.h5') 
emojis = {
    'Angry': '😡',
    'Disgust': '🤢',
    'Fear': '😨',
    'Happy': '😊',
    'Neutral': '😐',
    'Pleasant Surprise': '😮',
    'Sad': '😔'
}
USE_PIE_CHART = True

def show_prediction(predicted_emotion, predicted_emoji):
    loading_label.pack_forget()  
    predicted_emotion_text.set(f"Predicted Emotion: {predicted_emotion} {predicted_emoji}")
    loading_bar.stop() 
    
    
def extract_mfcc(filename):
    y, sr = librosa.load(filename, duration=3, offset=0.5)
    mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
    return mfcc, y, sr

def predict_emotion(audio_file):
    mfcc, _, _ = extract_mfcc(audio_file)
    mfcc = mfcc.reshape(1, 40, 1)
    predictions = model.predict(mfcc)
    return predictions[0]  # Return the probabilities for all emotions

    emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Pleasant Surprise', 'Sad']
    return emotions[predicted_label]

def plot_waveform(filename, sr, y):
    fig, ax = plt.subplots(figsize=(10, 6))  # Create a figure and axis
    ax.set_title('Audio Waveform')
    ax.plot(np.linspace(0, len(y) / sr, num=len(y)), y)
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Amplitude')
    ax.grid()
    return fig  # Return the figure


def record_and_process():
    def process_recorded_audio():
        file_path = "recorded_audio.wav"  # Get the recorded audio file path
        process_audio(file_path)  # Process the recorded audio for prediction

    try:
        global loading_label, loading_bar

        loading_label = tk.Label(root, text="Recording Audio...", font=("Arial", 14))
        loading_label.pack(pady=20)
        root.update()  # Update the GUI to show the label

        loading_bar = ttk.Progressbar(root, orient='horizontal', length=200, mode='indeterminate')
        loading_bar.pack(pady=10)

        def stop_recording():
            loading_label.config(text="Recording stopped")
            loading_bar.stop()

        # Record audio
        duration = 3  # Set the duration of recording in seconds
        fs = 44100  # Set the sampling frequency

        audio_data = sd.rec(int(duration * fs), samplerate=fs, channels=1)
        sd.wait()  # Wait until recording is finished

        file_path = "recorded_audio.wav"  # Define the file path to save the recorded audio
        sf.write(file_path, audio_data, fs)  # Save the recorded audio to a file
        loading_label.config(text="Recording finished")
        loading_bar.stop()

        root.after(1000, process_recorded_audio)  # Schedule processing of recorded audio after 1 second

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {str(e)}")




def remove_loading():
    loading_label.pack_forget()


def open_file():
    def process_recorded_audio():
        file_path = "recorded_audio.wav"  # Get the recorded audio file path
        process_audio(file_path)  # Process the recorded audio for prediction

    file_path = filedialog.askopenfilename()

    if file_path:
        try:
            global loading_label
            loading_label = tk.Label(root, text="Predicting Emotion...", font=("Arial", 14))
            loading_label.pack(pady=20)

            global loading_bar
            loading_bar = ttk.Progressbar(root, orient='horizontal', length=200, mode='indeterminate')
            loading_bar.pack(pady=10)
            loading_bar.start()

            # Check if a file is selected or audio needs to be recorded
            if file_path == "":
                # Record audio if no file is selected
                file_path = record_audio()
                # Schedule processing of recorded audio after 3 seconds
                root.after(3000, process_recorded_audio)
            else:
                # Process the selected audio file
                root.after(5000, lambda: process_audio(file_path))

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")

def process_audio(file_path):
    try:
        y, sr = librosa.load(file_path, sr=None)
        predicted_probabilities = predict_emotion(file_path)
        emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Pleasant Surprise', 'Sad']

        fig, (ax_waveform, ax_probs) = plt.subplots(1, 2, figsize=(14, 6))

        # Plot waveform
        ax_waveform.set_title('Audio Waveform')
        ax_waveform.plot(np.linspace(0, len(y) / sr, num=len(y)), y)
        ax_waveform.set_xlabel('Time (s)')
        ax_waveform.set_ylabel('Amplitude')
        ax_waveform.grid()

        # Plot emotion probabilities as a bar chart
        ax_probs.bar(emotions, predicted_probabilities, color='skyblue')
        ax_probs.set_xlabel('Emotions')
        ax_probs.set_ylabel('Probability')
        ax_probs.set_title('Emotion Probabilities')
        ax_probs.tick_params(axis='x', labelrotation=45)  # Rotate x-axis labels for better visibility

        # Show predicted emotion text on top of the window
        predicted_emotion = emotions[np.argmax(predicted_probabilities)]
        predicted_emoji = emojis.get(predicted_emotion, 'Unknown Emoji')
        predicted_emotion_text.set(f"Predicted Emotion: {predicted_emotion} {predicted_emoji}")

        fig.tight_layout()  # Adjust layout to prevent overlap

        # Pack the figure into a tkinter window
        canvas = FigureCanvasTkAgg(fig, master=root)
        canvas.get_tk_widget().pack(padx=10, pady=10)
        canvas.draw()

        show_prediction(predicted_emotion, predicted_emoji)

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {str(e)}")
        



            
            
def create_gui():
    global root
    root = tk.Tk()
    root.title("Emotion Predictor")
    root.geometry("800x600")  # Set a fixed size for the window

 
    signal_processing_heading = tk.Label(root, text="Signal Processing", font=("Arial", 18))
    signal_processing_heading.pack()

    # Create a frame for the file selection and predicted text
    file_frame = tk.Frame(root)
    file_frame.pack(pady=20)

    # Add the predicted emotion text with larger font size
    global predicted_emotion_text
    predicted_emotion_text = tk.StringVar()
    predicted_emotion_label = tk.Label(file_frame, textvariable=predicted_emotion_text, font=("Arial", 31))
    predicted_emotion_label.pack()

    button = tk.Button(file_frame, text="Select Audio File", command=open_file)
    button.pack(padx=20, pady=10)
    
    button = tk.Button(file_frame, text="Record Audio", command=record_and_process)
    button.pack(padx=20, pady=10)

    # Create a frame for the plot and emojis
    plot_frame = tk.Frame(root)
    plot_frame.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

    root.mainloop()


if __name__ == "__main__":
    create_gui()


1/1 [==============================] - 0s 364ms/step
